In [1]:
import syft as sy
sy.requires(">=0.8.1-beta")
sy.enable_autoreload()

✅ The installed version of syft==0.8.1b1 matches the requirement >=0.8.1b0
Autoreload enabled


In [2]:
domain_1 = sy.orchestra.launch(name="test-domain-1", dev_mode=True, reset=True)

SQLite Store Path:
!open file:///var/folders/6_/7xx0tpq16h9cn40mq4w5gjk80000gn/T/7bca415d13ed1ec841f0d0aede098dbb.sqlite

> Domain: test-domain-1 - 7bca415d13ed1ec841f0d0aede098dbb - NodeType.DOMAIN

Services:
ActionService, DataSubjectMemberService, DataSubjectService, DatasetService, MessageService, MetadataService, NetworkService, NewProjectService, PolicyService, ProjectService, RequestService, UserCodeService, UserService


In [3]:
domain_1_client = domain_1.login(email="info@openmined.org", password="changethis")

In [4]:
domain_2 = sy.orchestra.launch(name="test-domain-2", dev_mode=True, reset=True)

SQLite Store Path:
!open file:///var/folders/6_/7xx0tpq16h9cn40mq4w5gjk80000gn/T/b626124c319383c882a448dac2d6953d.sqlite

> Domain: test-domain-2 - b626124c319383c882a448dac2d6953d - NodeType.DOMAIN

Services:
ActionService, DataSubjectMemberService, DataSubjectService, DatasetService, MessageService, MetadataService, NetworkService, NewProjectService, PolicyService, ProjectService, RequestService, UserCodeService, UserService


In [5]:
domain_2_client = domain_2.login(email="info@openmined.org", password="changethis")

In [6]:
domain_1_client.exchange_route(domain_2_client)

<class 'syft.service.response.SyftSuccess'>: Route Verified

In [7]:
domain_1_client.domains

,type,id,name
0,syft.service.network.network_service.NodePeer,b626124c319383c882a448dac2d6953d,test-domain-2


In [8]:
assert len(domain_1_client.domains) == 1

In [9]:
domain_2_client.domains

,type,id,name
0,syft.service.network.network_service.NodePeer,7bca415d13ed1ec841f0d0aede098dbb,test-domain-1


In [10]:
assert len(domain_2_client.domains) == 1

In [11]:
from syft.service.project.project import NewProjectSubmit
from syft.service.project.project import NewProject
from syft.service.project.project import DemocraticConsensusModel
from syft.service.metadata.node_metadata import NodeMetadata

In [12]:
project = NewProjectSubmit(
    name="test project",
    description="my test project",
    shareholders=[domain_1_client, domain_2_client],
    consensus_model=DemocraticConsensusModel(),
)
project

Got metadata syft.service.metadata.node_metadata.NodeMetadata
got node identity,  <🔑 aec6ea4d @ 🟢 7bca415d> <class 'syft.service.project.project.NodeIdentity'>
Got metadata syft.service.metadata.node_metadata.NodeMetadata
got node identity,  <🔑 0dfade1d @ 🟢 b626124c> <class 'syft.service.project.project.NodeIdentity'>


```python
class NewProjectSubmit:
  id: str = None
  name: str = "test project"
  description: str = "my test project"
  shareholders: str = [<🔑 aec6ea4d @ 🟢 7bca415d>, <🔑 0dfade1d @ 🟢 b626124c>]
  project_permissions: str = set()
  state_sync_leader: str = None
  consensus_model: str = <syft.service.project.project.DemocraticConsensusModel object at 0x1779f2680>

```

In [13]:
results = project.start()
results

,type,id,name,shareholders,state_sync_leader
0,syft.service.project.project.NewProject,4944aad2ebd4460fb003e35bf9f7abfb,test project,"[<🔑 aec6ea4d @ 🟢 7bca415d>, <🔑 0dfade1d @ 🟢 b6...",<🔑 aec6ea4d @ 🟢 7bca415d>
1,syft.service.project.project.NewProject,4944aad2ebd4460fb003e35bf9f7abfb,test project,"[<🔑 aec6ea4d @ 🟢 7bca415d>, <🔑 0dfade1d @ 🟢 b6...",<🔑 aec6ea4d @ 🟢 7bca415d>


In [ ]:
results[0].start_hash

In [ ]:
assert results[0].start_hash == results[1].start_hash

In [ ]:
project = results[0]

In [ ]:
project.shareholders

In [ ]:
assert isinstance(project, NewProject)

In [ ]:
assert len(domain_1_client.api.services.newproject.get_all()) == 1

## Project Messaging

In [ ]:
from syft.service.project.project import ProjectMessage

In [ ]:
prev_hash = results[0].start_hash

In [ ]:
message = "My cool message"
message_event = ProjectMessage(
    project_id=project.id,
    creator_verify_key=domain_1_client.credentials.verify_key,
    prev_event_hash=prev_hash,
    message=message
)

In [ ]:
message_event

In [ ]:
message_event.event_hash = hash(message_event) + prev_hash

In [ ]:
message_event.sign(domain_1_client.credentials)

In [ ]:
message_event

In [ ]:
message_event.valid

In [ ]:
project.add_event(message_event)

In [ ]:
domain_1_client.api.services.newproject.get_all()[0]

In [ ]:
domain_2_client.api.services.newproject.get_all()[0]

## Dataset Upload

In [ ]:
import numpy as np
dataset = sy.Dataset(name="Domains's... Private Data")
asset = sy.Asset(name="Secret data")

# Real Data
x = np.array([1, 2, 3])
asset.set_obj(x)

# Mock Data
y = np.array([1, 1, 1])
asset.set_mock(y, mock_is_real=False)
dataset.add_asset(asset)

In [ ]:
domain_1_client.upload_dataset(dataset)

In [ ]:
domain_1_client.datasets

In [ ]:
domain_1_dataset = domain_1_client.datasets[-1]
@sy.syft_function(
    input_policy=sy.ExactMatch(
        input_data=domain_1_dataset.assets[0]
    ),
    output_policy=sy.SingleExecutionExactOutput(),
)
def simple_function(input_data):

    return input_data+input_data

In [ ]:
simple_function

In [ ]:
# req = domain_1_client.api.services.code.request_code_execution(simple_function)

In [ ]:
# updated_proj = project.add_event(message_event)

In [ ]:
# updated_proj

In [ ]:
domain_1_client.api.services.newproject.get_all()[0]

In [ ]:
domain_2_client.api.services.newproject.get_all()[0]

In [ ]:
for node in [domain_1, domain_2]:
    if node.node_type.value == "python":
        node.land()